In [78]:
import pandas as pd
pd.set_option('display.max_columns', 100)
%matplotlib inline
# 2013-2014 loan data - files are available from deeper history
past_loans = pd.read_csv('LoanStats3c_securev1.csv', header = 1)
past_loans['term'] = past_loans['term'].astype('category')
past_loans['grade'] = past_loans['grade'].astype('category')
past_loans['sub_grade'] = past_loans['sub_grade'].astype('category')
past_loans['home_ownership'] = past_loans['home_ownership'].astype('category')
past_loans['verification_status'] = past_loans['verification_status'].astype('category')
past_loans['loan_status'] = past_loans['loan_status'].astype('category')
past_loans['pymnt_plan'] = past_loans['pymnt_plan'].astype('category')
past_loans['purpose'] = past_loans['purpose'].astype('category')
past_loans['application_type'] = past_loans['application_type'].astype('category')
past_loans['issue_d'] = pd.to_datetime(past_loans['issue_d'])
past_loans['earliest_cr_line'] = pd.to_datetime(past_loans['earliest_cr_line'])
past_loans['last_pymnt_d'] = pd.to_datetime(past_loans['last_pymnt_d'])
past_loans['next_pymnt_d'] = pd.to_datetime(past_loans['next_pymnt_d'])
past_loans['last_credit_pull_d'] = pd.to_datetime(past_loans['last_credit_pull_d'])
past_loans['issue_d'] = pd.to_datetime(past_loans['issue_d'])
past_loans=past_loans.dropna(axis=1,how='all')
past_loans.drop('application_type',axis = 1) # only contains "individual"

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_credit_rv
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.00,Not Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0,1989-09-01,710,714,2,42,NaN,17,0,6133,31.6%,36,w,7449.57,7449.57,3521.780000,3521.78,2950.43,571.35,0.00,0.00,0.0000,2015-12-01,321.08,2016-01-01,2015-12-01,709,705,0,59,1,0,0,162110,19400
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.00,Source Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0,1994-08-01,750,754,0,NaN,NaN,6,0,138008,29%,17,w,12894.10,12894.10,3691.360000,3691.36,2105.90,1585.46,0.00,0.00,0.0000,2015-12-01,336.64,2016-02-01,2015-12-01,684,680,0,NaN,1,0,0,149140,184500
2,37612354,40375473,12800,12800,12800,60 months,17.14%,319.08,D,D4,Senior Sales Professional,10+ years,MORTGAGE,125000.00,Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,car,Car financing,953xx,CA,8.31,1,2000-10-01,665,669,0,17,NaN,8,0,5753,100.9%,13,w,11189.45,11189.45,3479.410000,3479.41,1610.55,1868.86,0.00,0.00,0.0000,2015-12-01,319.08,2016-01-01,2015-12-01,734,730,0,36,1,0,0,261815,5700
3,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,Admin Specialist,10+ years,RENT,69000.00,Source Verified,2014-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,077xx,NJ,25.81,0,1992-11-01,680,684,0,NaN,NaN,12,0,16388,59.4%,44,f,0.00,0.00,9973.430000,9973.43,9600.00,373.43,0.00,0.00,0.0000,2015-04-01,9338.58,NaT,2015-12-01,684,680,0,NaN,1,0,0,38566,27600
4,37842129,40605224,21425,21425,21425,60 months,15.59%,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.00,Source Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,658xx,MO,18.49,0,2003-08-01,685,689,0,60,NaN,10,0,16374,76.2%,35,w,18629.98,18629.98,5633.570000,5633.57,2795.02,2838.55,0.00,0.00,0.0000,2015-12-01,516.36,2016-01-01,2015-12-01,684,680,0,74,1,0,0,42315,21500
5,37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000.00,Source Verified,2014-12-01,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0,2002-08-01,685,689,1,NaN,NaN,11,0,16822,91.9%,20,f,0.00,0.00,1043.990000,1043.99,704.38,339.61,0.00,0.00,0.0000,2015-08-01,17.70,NaT,2015-12-01,539,535,0,NaN,1,0,0,64426,18300
6,37800722,40563521,12975,12975,12975,36 months,17.86%,468.17,D,D5,Sales,10+ years,RENT,60000.00,Source Verified,2014-12-01,Late (31-120 days),n,https://www.lendingclub.com/browse/loanDetail....,NaN,house,Home buying,331xx,FL,22.42,0,1999-01-01,680,684,0,48,NaN,11,0,5200,33.1%,19,f,10346.86,10346.86,4181.340000,4181.34,2628.14,1553.20,0.00,0.00,0.0000,2015-10-01,468.17,2016-01-01,2015-12-01,634,630,0,NaN,1,0,900,17281,15700
7,37701596,40474581,10000,10000,10000,36 months,11.99%,332.10,B,B5,Investment Consultant,8 years,RENT,90000.00,Verified,2014-12-01,Cu

In [79]:
pd.unique(past_loans['term'])

array([' 36 months', ' 60 months', nan], dtype=object)

In [80]:
pd.unique(past_loans['grade'])

array(['A', 'C', 'D', 'B', 'E', 'F', 'G', nan], dtype=object)

In [81]:
pd.unique(past_loans['sub_grade'])

array(['A3', 'C1', 'D4', 'C3', 'D1', 'D5', 'B5', 'B4', 'C4', 'E5', 'D2',
       'B3', 'C5', 'E4', 'E3', 'C2', 'B2', 'A5', 'F1', 'B1', 'D3', 'A4',
       'E1', 'E2', 'G2', 'A1', 'G1', 'A2', 'F3', 'F2', 'G3', 'F4', 'G4',
       'F5', 'G5', nan], dtype=object)

In [82]:
pd.unique(past_loans['home_ownership'])

array(['MORTGAGE', 'RENT', 'OWN', 'ANY', nan], dtype=object)

In [83]:
pd.unique(past_loans['verification_status'])

array(['Not Verified', 'Source Verified', 'Verified', nan], dtype=object)

In [84]:
pd.unique(past_loans['loan_status'])

array(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)', 'Default', nan], dtype=object)

In [85]:
pd.unique(past_loans['pymnt_plan'])

array(['n', 'y', nan], dtype=object)

In [86]:
pd.unique(past_loans['purpose'])

array(['credit_card', 'debt_consolidation', 'car', 'house',
       'home_improvement', 'other', 'medical', 'moving', 'major_purchase',
       'vacation', 'small_business', 'renewable_energy', 'wedding', nan], dtype=object)

In [87]:
past_loans.head(100)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_credit_rv
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000,Not Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0,1989-09-01,710,714,2,42,NaN,17,0,6133,31.6%,36,w,7449.57,7449.57,3521.78,3521.78,2950.43,571.35,0,0.00,0.0000,2015-12-01,321.08,2016-01-01,2015-12-01,709,705,0,59,1,INDIVIDUAL,0,0,162110,19400
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000,Source Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0,1994-08-01,750,754,0,NaN,NaN,6,0,138008,29%,17,w,12894.10,12894.10,3691.36,3691.36,2105.90,1585.46,0,0.00,0.0000,2015-12-01,336.64,2016-02-01,2015-12-01,684,680,0,NaN,1,INDIVIDUAL,0,0,149140,184500
2,37612354,40375473,12800,12800,12800,60 months,17.14%,319.08,D,D4,Senior Sales Professional,10+ years,MORTGAGE,125000,Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,car,Car financing,953xx,CA,8.31,1,2000-10-01,665,669,0,17,NaN,8,0,5753,100.9%,13,w,11189.45,11189.45,3479.41,3479.41,1610.55,1868.86,0,0.00,0.0000,2015-12-01,319.08,2016-01-01,2015-12-01,734,730,0,36,1,INDIVIDUAL,0,0,261815,5700
3,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,Admin Specialist,10+ years,RENT,69000,Source Verified,2014-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,077xx,NJ,25.81,0,1992-11-01,680,684,0,NaN,NaN,12,0,16388,59.4%,44,f,0.00,0.00,9973.43,9973.43,9600.00,373.43,0,0.00,0.0000,2015-04-01,9338.58,NaT,2015-12-01,684,680,0,NaN,1,INDIVIDUAL,0,0,38566,27600
4,37842129,40605224,21425,21425,21425,60 months,15.59%,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800,Source Verified,2014-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,658xx,MO,18.49,0,2003-08-01,685,689,0,60,NaN,10,0,16374,76.2%,35,w,18629.98,18629.98,5633.57,5633.57,2795.02,2838.55,0,0.00,0.0000,2015-12-01,516.36,2016-01-01,2015-12-01,684,680,0,74,1,INDIVIDUAL,0,0,42315,21500
5,37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000,Source Verified,2014-12-01,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,850xx,AZ,34.81,0,2002-08-01,685,689,1,NaN,NaN,11,0,16822,91.9%,20,f,0.00,0.00,1043.99,1043.99,704.38,339.61,0,0.00,0.0000,2015-08-01,17.70,NaT,2015-12-01,539,535,0,NaN,1,INDIVIDUAL,0,0,64426,18300
6,37800722,40563521,12975,12975,12975,36 months,17.86%,468.17,D,D5,Sales,10+ years,RENT,60000,Source Verified,2014-12-01,Late (31-120 days),n,https://www.lendingclub.com/browse/loanDetail....,NaN,house,Home buying,331xx,FL,22.42,0,1999-01-01,680,684,0,48,NaN,11,0,5200,33.1%,19,f,10346.86,10346.86,4181.34,4181.34,2628.14,1553.20,0,0.00,0.0000,2015-10-01,468.17,2016-01-01,2015-12-01,634,630,0,NaN,1,INDIVIDUAL,0,900,17281,15700
7,37701596,40474581,10000,10000,10000,36 months,11.99%,332.10,B,B5,Investment Consultant,8 years,RENT,90000,V

In [88]:
past_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235631 entries, 0 to 235630
Data columns (total 61 columns):
id                             235631 non-null object
member_id                      235629 non-null float64
loan_amnt                      235629 non-null float64
funded_amnt                    235629 non-null float64
funded_amnt_inv                235629 non-null float64
term                           235629 non-null category
int_rate                       235629 non-null object
installment                    235629 non-null float64
grade                          235629 non-null category
sub_grade                      235629 non-null category
emp_title                      222393 non-null object
emp_length                     235629 non-null object
home_ownership                 235629 non-null category
annual_inc                     235629 non-null float64
verification_status            235629 non-null category
issue_d                        235629 non-null datetime64[ns]
l

In [89]:
past_loans.describe().transpose() #.to_string()

,count,mean,std,min,25%,50%,75%,max
member_id,235629,24019299.796719,8825162.024128,137225.00,15579724.00,22953173.00,31767065.00,40860827.0000
loan_amnt,235629,14870.156793,8438.318193,1000.00,8325.00,13000.00,20000.00,35000.0000
funded_amnt,235629,14870.156793,8438.318193,1000.00,8325.00,13000.00,20000.00,35000.0000
funded_amnt_inv,235629,14865.334169,8435.524995,950.00,8325.00,13000.00,20000.00,35000.0000
installment,235629,442.482374,245.050238,23.36,265.68,384.12,578.71,1409.9900
annual_inc,235629,74854.148281,55547.533374,3000.00,45377.00,65000.00,90000.00,7500000.0000
dti,235629,18.040770,8.023002,0.00,12.02,17.63,23.76,39.9900
delinq_2yrs,235629,0.344512,0.898319,0.00,0.00,0.00,0.00,22.0000
fico_range_low,235629,692.497358,29.246641,660.00,670.00,685.00,705.00,845.0000
fico_range_high,235629,696.497418,29.246952,664.00,674.00,689.00,709.00,850.0000
